In [5]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-lqa4ni8j
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-lqa4ni8j
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=82e09824c9535cee83fa772ab4fa2b0ef788b252fdba8735f945d9b29918a362
  Stored in directory: /tmp/pip-ephem-wheel-cache-h9p5tinc/wheels/c3/03/25/5e0ba78bc27a3a089f137c9f1d92fdfce16d06996c071a016c
Successfully built openai-whisper


In [6]:

!pip install torch

In [7]:
!pip install soundfile

In [8]:
!apt-get update && apt-get install -y ffmpeg


Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://cli.github.com/packages stable/main amd64 Packages [343 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,149 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy

In [9]:
!pip install git+https://github.com/openai/whisper.git


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-tpc_9pno
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-tpc_9pno
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


**Text to Speech**

In [10]:
!pip install gTTS

In [11]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118


*** Audio Pipeline***

In [13]:
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import FileResponse
import uvicorn
import uuid
import os
import whisper
from gtts import gTTS

app = FastAPI(title="Audio Pipeline API (STT + TTS)")

# -------- Whisper Model --------
model = whisper.load_model("small")

# ----------- STT ---------------
@app.post("/stt")
async def speech_to_text(file: UploadFile = File(...)):
    temp_name = f"input_{uuid.uuid4()}.mp3"
    with open(temp_name, "wb") as f:
        f.write(await file.read())

    result = model.transcribe(temp_name, language="ar")
    text = result["text"]

    os.remove(temp_name)
    return {"text": text}

# ----------- TTS ---------------
@app.post("/tts")
async def text_to_speech(text: str = Form(...)):
    audio_name = f"tts_{uuid.uuid4()}.mp3"
    tts = gTTS(text=text, lang="ar")
    tts.save(audio_name)

    return FileResponse(audio_name, media_type="audio/mpeg", filename=audio_name)


# ----------- Run Server ----------
if __name__ == "__main__":
    uvicorn.run("app:app", host="0.0.0.0", port=8000, reload=True)


100%|████████████████████████████████████████| 461M/461M [00:03<00:00, 125MiB/s]
INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [233] using StatReload
INFO:     Stopping reloader process [233]
